## Watusi Legal Insight

### Your First STop for Legal Clarity

## Load Libraries

In [391]:
#!pip install chromadb
# !pip install ragas


from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
import os
import regex as re
from datetime import datetime
from typing import TypedDict, List, Dict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import NotRequired
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, START, END
from langchain.schema import Document
from langchain_core.messages import AIMessage,HumanMessage, SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder)
from langchain_ollama import OllamaLLM
from langchain_openai import ChatOpenAI
import json
from langchain_chroma import Chroma
import chromadb
from langchain_mistralai import MistralAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank
from mistralai import Mistral
from langchain_mistralai.chat_models import ChatMistralAI
import uuid

#rag evaluation's library
import random
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.llms import LangchainLLMWrapper

import warnings
warnings.filterwarnings('ignore')

## Load Documents and Convert to String

In [3]:
folder_path = 'documents'

noise_patterns = [
    r'www\.hukumonline\.com',
    r'\nMenemukan kesalahan ketik dalam dokumen\?\nKlik di sini\nuntuk perbaikan.'
]
# pattern = r'(www\.hukumonline\.com|\nMenemukan kesalahan ketik dalam dokumen\?\nKlik di sini\nuntuk perbaikan.)'
all_docs = []

for file in os.listdir(folder_path):
    if file.endswith('.pdf'):
        file_path = os.path.join(folder_path, file)
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        for doc in docs:
            for noise in noise_patterns:
                doc.page_content = re.sub(noise, '', doc.page_content)
        
        all_docs.extend(docs)

full_text = ''.join(page.page_content for page in all_docs)

## Parse The Document into Customize Documents

In [ ]:
list_buku = []
list_bab = []
docs = []
buku_chunks = re.split(r'(?=BUKU\s+KE\w+)', full_text)
for buku in buku_chunks:
    buku_pattern = r'(BUKU\s+KE\w+\s)([\s\S]*)'
    buku_match = re.search(buku_pattern, buku, re.DOTALL)
    # if not re.search(buku_pattern, buku, re.DOTALL):
    if not buku_match:
        continue
    list_buku.append(buku_match.group(1).strip())
    bab_chunks = re.split(r'(?=BAB\s+[IVXLCDM]+)', buku)
    for bab in bab_chunks:
        bab_pattern =  r'(BAB\s+[IVXLCDM]+[A-Z]?)\s*\n(.*)\s*\n([\s\S]*)' 
        bab_match = re.search(bab_pattern, bab)
        # bab_pattern =  r'(BAB\s+[IVXLCDM]+)\s*\n(.*)\s*\n(.*)'
        # bab_match = re.search(bab_pattern, bab, re.DOTALL)
        if not bab_match:
            continue
        bab_number = bab_match.group(1).strip()
        bab_title = bab_match.group(2).strip().replace('\n','')
        bab_content = bab_match.group(3).strip()

        # print(f'{buku_match.group(1)} -> {bab_number}, {bab_title}')

        # Proses pasal pasal yang ada di level "BAB"
        if 'BAGIAN' in bab:
            before_bagian = re.split(r'(?=BAGIAN\s+[0-9]+)', bab)[0]
        
        else:
            before_bagian = bab
        pasal_chunks = re.split(r'(?=Pasal\s+[0-9]+[a-z]?)',before_bagian)
        for pasal in pasal_chunks:
            pasal_pattern = r'(Pasal\s+[0-9]+[a-z]?)\s*\n(.*)'
            pasal_match = re.search(pasal_pattern, pasal, re.DOTALL)
            if not pasal_match:
                continue
            pasal_number = pasal_match.group(1).lower().replace('pasal','').strip()
            pasal_content = pasal_match.group(2).lower().replace('\n','').strip()
            
            doc = Document(
                        page_content =
                            f"Bab: {bab_number} - {bab_title}\n"
                            f"Pasal: {pasal_number}\n"
                            f"Isi Pasal: {pasal_content}",
                        metadata = {
                        'document_type': 'legal_document',
                        'document_title': buku_match.group(1),
                        'bab_number': bab_number,
                        'bab_title': bab_title,
                        'bagian': '',
                        'bagian_title': '',
                        'pasal': pasal_number})
            docs.append(doc)

            print(f'{buku_match.group(1)} -> {bab_number}, {bab_title} -> {pasal_number}')

        #Proses pasal pasal yang ada di level "BAGIAN"
        if 'BAGIAN' in bab:
            bagian_chunks = re.split(r'(?=BAGIAN\s+[0-9]+)', bab)
            for bagian in bagian_chunks:
                bagian_pattern = r'(BAGIAN\s+[0-9]+)\s*\n*(.*)\s+([\s\S]+)'
                bagian_match = re.search(bagian_pattern, bagian)
                # bagian_pattern = r'(BAGIAN\s+[0-9]+)\s*\n*(.*)\s+(.*)'
                # bagian_match = re.search(bagian_pattern, bagian, re.DOTALL)
                if not bagian_match:
                    continue
                bagian_number = bagian_match.group(1).strip()
                bagian_title = bagian_match.group(2).strip().replace('\n','')
                # bagian_content = bagian_match.group(3).strip()
                
                # print(f'{buku_match.group(1)} -> {bab_number}, {bab_title} -> {bagian_number}, {bagian_title}')

                pasal_chunks = re.split(r'(?=Pasal\s+[0-9]+[a-z]?)',bagian)
                for pasal in pasal_chunks:
                    pasal_pattern = r'Pasal\s+([0-9]+[a-z]?)\s*\n(.*)'
                    pasal_match = re.search(pasal_pattern, pasal, re.DOTALL)
                    if not pasal_match:
                        continue
                    # pasal_number = pasal_match.group(1).strip()
                    pasal_number = pasal_match.group(1).lower().replace('pasal','').strip()
                    pasal_content = pasal_match.group(2).lower().replace('\n','').strip()
                    doc = Document(
                        page_content =
                            f"Bab: {bab_number} - {bab_title}\n"
                            f"Bagian: {bagian_number} - {bagian_title}\n"
                            f"Pasal: {pasal_number}\n"
                            f"Isi Pasal: {pasal_content}",
                        metadata = {
                        'document_type': 'legal_document',
                        'document_title': buku_match.group(1),
                        'bab_number': bab_number,
                        'bab_title': bab_title,
                        'bagian': bagian_number,
                        'bagian_title': bagian_title,
                        'pasal': pasal_number})
                    docs.append(doc)

                    # print(f'{buku_match.group(1)} -> {bab_number}, {bab_title} -> {pasal_number}')


                    # print(f'{buku_match.group(1)} -> {bab_number}, {bab_title} -> {bagian_number}, {bagian_title}')
                # print(f'{pasal_number, pasal_content}')
                


In [332]:
#document evaluation
print([doc for doc in docs if doc.metadata['bab_title'] == 'PEMBUKTIAN PADA UMUMNYA'][0].page_content)

Bab: BAB I - PEMBUKTIAN PADA UMUMNYA
Pasal: 1865
Isi Pasal: setiap orang yang mengaku mempunyai suatu hak, atau menunjuk suatu peristiwa untuk meneguhkan haknya itu atau untuk membantah suatu hak orang lain, wajib membuktikan adanya hak itu atau kejadian yang dikemukakan itu.


## RAG Pipeline

### Embed Documents, Store in Vector DB and Rerank with Cross-Encoder

In [336]:
#load api key
with open('credentials.json', 'r')as f:
    data = json.load(f)

mistral_api = data['mistral_api']
jina_api = data['jina_api']
#------------------------------------------------------------------------------------------------------------------------------------------------------------

#load embedding model
model_name = 'mistral-embed'
embedding_model = MistralAIEmbeddings(
    model = model_name,
    api_key=mistral_api)
#------------------------------------------------------------------------------------------------------------------------------------------------------------

#store / load document from chroma db
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='.\pipeline\chroma_db',
    collection_name = 'kuhperdata_mistral'   
)
# vector_store.add_documents(docs) #-> Run this script to add document to vector db
#------------------------------------------------------------------------------------------------------------------------------------------------------------

#set vector db as retriver with search type similarity and get 'top 5' documents
# retriever = vector_store.as_retriever(
#     search_type = 'similarity',
#     k= 5)

retriever = vector_store.as_retriever(
    search_type = 'mmr',
    search_kwargs = {
        'fetch_k': 10,
        'k': 3})

#rerank the documents with the help of Cross Encoder model (in my case I'm utilazing Jina Reranker model)
compressor = JinaRerank(jina_api_key = jina_api, top_n = 2)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever)
#------------------------------------------------------------------------------------------------------------------------------------------------------------

# Run code below to identified what vector db (collection) that available in the local storage
# client = chromadb.PersistentClient(path = './pipeline/chroma_db/')
# client.list_collections()

In [335]:
# client = chromadb.PersistentClient(path = './pipeline/chroma_db/')
# client.list_collections()
# client.delete_collection(name = 'kuhperdata_mistral')

In [9]:
def debug_retrieval(retriever, query, expected_content=None):
    """Debug retrieval dengan eye-balling results"""
    results = retriever.similarity_search_with_score(query, k=3)
    
    print(f"Query: {query}")
    print("=" * 50)
    
    for i, (doc, score) in enumerate(results):
        print(f"Rank {i+1} | Score: {score:.4f}")
        print(f"Content: {doc.page_content[:200]}...")
        if expected_content and expected_content.lower() in doc.page_content.lower():
            print("✅ CONTAINS EXPECTED CONTENT!")
        print("-" * 30)

# Test
# debug_retrieval(vector_store, "Pasal 1234", "Perikatan ditujukan untuk memberikan sesuatu, untuk berbuat sesuatu, atau untuk tidak berbuat sesuatu")

### Load LLM Model

1. Mistral LLM for RAG Chain -> *gemma model as alternative with local environment*
2. OpenRouter API to Access LLM Model (For RAG Evaluation)

In [394]:
with open('credentials.json', 'r')as f:
    data = json.load(f)

mistral_api = data['mistral_api']
open_route_api = data['open_route_api']
genai_api = data['google_ai_api']

#load gemma model
ollama_llm = OllamaLLM(
    model = 'gemma3:1b',
    temperature = 0.2)

#load mistral model
mistral_llm = ChatMistralAI(
    # model = 'Hermes-2-Pro-Mistral-7B',
    model = 'mistral-small-2506',
    api_key=mistral_api,
    temperature = 0.1
)

#load gemini model
openrouter_llm = ChatOpenAI(
    model = 'openai/gpt-oss-120b:free',
    base_url="https://openrouter.ai/api/v1",
    api_key = open_route_api
)

# genai_llm = ChatGoogleGenerativeAI(
#     model = 'gemini-2.5-flash-lite',
#     api_key = genai_api)

### Create RAG Chain Utilizing Langchain and Langgraph

In [12]:
greeting_keywords = [
# Umum
"halo", "hai", "hei", "apa kabar", "bagaimana kabarnya", "salam",
# Berdasarkan waktu
"selamat pagi", "selamat siang", "selamat sore", "selamat malam", "selamat subuh",
"pagi", "siang", "sore", "malam",

# Gaul / Santai
"bro", "sis", "cuy", "sob", "gan",

# Formal / Sopan
"assalamu’alaikum", "wa’alaikumussalam", "salam sejahtera",
"om swastiastu", "shalom", "namo buddhaya", "salam kebajikan"]

legal_keywords = [
# Dasar Hukum / Aturan
"undang-undang", "uu", "peraturan", "perpu", "pp", "permen", "putusan", "putusan mahkamah", 
"putusan pengadilan", "perda", "perkap", "keputusan", "instruksi", "ketetapan",

# Pasal & Kitab Hukum
"pasal", "ayat", "kitab undang-undang hukum pidana", "kuhp", "kitab undang-undang hukum perdata",
"kuhperdata", "kuhap", "uu ite", "uu perkawinan", "uu ketenagakerjaan",

# Proses Peradilan
"hakim", "jaksa", "pengacara", "penasihat hukum", "tersangka", "terdakwa", "saksi", "korban",
"sidang", "peradilan", "putusan", "putusan kasasi", "putusan banding", "putusan mk", "putusan ma",

# Jenis Hukum
"hukum pidana", "hukum perdata", "hukum tata negara", "hukum administrasi", 
"hukum adat", "hukum islam", "hukum dagang",

# Istilah Kontrak / Perjanjian
"kontrak", "perjanjian", "akta", "notaris", "hak dan kewajiban", "sanksi", "gugatan",
"wanprestasi", "perbuatan melawan hukum",

# Istilah Pidana Umum
"korupsi", "pencurian", "penggelapan", "penipuan", "pembunuhan", "pemerkosaan",
"pencucian uang", "narkotika", "terorisme",

# Istilah Sengketa / Arbitrase
"sengketa", "arbitrase", "mediasi", "negosiasi", "perdamaian", "putusan arbitrase"
]

followup_keywords = ['lebih detail', 'maksudnya', 'contohnya', 'jelaskan lagi', 
                        'bagaimana dengan', 'lalu', 'kemudian', 'selanjutnya']

In [275]:
class OverallState(TypedDict):
    question: str
    retrieved_docs: List[Document]
    prompt: str
    messages: Annotated[List[BaseMessage], add_messages]
    llm: object
    nama: NotRequired[str]
    domisili: NotRequired[str]
    summary: NotRequired[str]

def classify_query(state: OverallState) -> dict:
    input = state['question']
    if any(word in input.lower() for word in greeting_keywords):
        return 'greeting'
    # elif any(word in input.lower() for word in legal_keywords):
    elif 'pasal' in input.lower():
        return 'legal_specific'
    elif any(word in input.lower() for word in followup_keywords):
        return 'followup'
    return 'general'


def retriever_node(state: OverallState) -> dict:
    question_type = classify_query(state)
    question = state['question']
    if question_type == 'greeting':
        return {
        **state,
        'retrieved_docs': []}
    elif question_type == 'legal_specific':
        try:
            matches = re.findall(r'(?i)(?<=pasal\s*)\d+\s*[a-z]?\b', question)
            if matches:
                docs = []

                # Batch process 
                retriever_pasal = vector_store.as_retriever(
                    search_type = 'similarity',
                    search_kwargs = {'k':1})
                for match in matches:
                    pasal_key = match.replace(' ','')
                    print(f"📋 [RETRIEVER NODE] Processing pasal: {pasal_key}")
                    try:
                        result = retriever_pasal.get_relevant_documents(pasal_key, filter = {'pasal': pasal_key})
                        docs.extend(result)
                    except Exception as e:
                        print(f"❌ [RETRIEVER NODE] Error retrieving pasal {pasal_key}: {e}")

            else:
                print("🔍 [RETRIEVER NODE] No pasal found")
                docs = []
        except Exception as e:
            print(f"❌ [RETRIEVER NODE] Error: {e}")
            docs = []   
        return {
            **state,
            'retrieved_docs': docs
        }
    else:
        return {
            **state,
            'retrieved_docs': []
        }

def merger_node(state:OverallState) -> dict:
    summary = state.get('summary','')
    docs = state.get('retrieved_docs', [])
    context = '\n'.join([doc.page_content for doc in state['retrieved_docs']])
    system_prompt = f"""
    Kamu adalah seorang ahli hukum. Jawab pertanyaan user berdasarkan **Konteks berikut**:

    Ringkasan Percakapan Sebelumnya:
    {summary}

    Dokumen relevan:
    {context}

    
    ### Instruksi:
    - Jawab selalu dalam **bahasa Indonesia** dan **output harus di bawah 50 kata**
    - **Jawaban langsung ke inti**
    - Jika pertanyaannya meminta isi pasal (misalnya menyebut "Pasal 1320", "bunyi Pasal", dsb), kutip isi pasal yang diminta **secara langsung** dari konteks.
    - Jika pertanyaannya berupa kasus hukum (contoh: pelanggaran perjanjian kerja sama, wanprestasi, dll), berikan pasal **yang relevan saja** dari konteks, jangan mengarang.
    - Jika informasi tidak ditemukan dalam konteks, katakan bahwa **informasi tidak tersedia** atau **ajukan pertanyaan untuk meminta kejelasan ke user**
    
    ### Deteksi Lanjutan (Sinyal Eskalasi):
    Jika user menunjukkan salah satu dari hal berikut, barulah berikan **kontak pengacara**:
    1. Menyatakan **bingung secara hukum**, misal: “Saya tidak tahu harus bagaimana”, “Tolong bantu saya”
    2. Menyatakan ingin **dibantu secara langsung**
    3. Menyebut butuh bantuan **membuat/review dokumen**
    4. Menunjukkan **keputusasaan/kebingungan yang jelas**
    5. Menyatakan ingin **berkonsultasi lebih lanjut**
    6. Jangan berikan tawaran bantuan hukum **di awal percakapan**

    ### Jika Sinyal Terdeteksi:
    Tambahkan di akhir jawaban:
    "Jika Anda membutuhkan bantuan lebih lanjut, silakan hubungi kami melalui email [Watusi Legal Insight](contact@watusilegalinsight.com) atau WhatsApp 0812-3456-7890. Tim kami akan menghubungi Anda setelah verifikasi data dilakukan."

    **Jangan berikan kontak jika belum ada sinyal di atas.**
    """
    prompt = ChatPromptTemplate([
        ('system', system_prompt),
        # ('human', state['question'])
        MessagesPlaceholder(variable_name = 'messages')
    ])
    
    return{
        **state,
        'prompt': prompt,
        'messages': [HumanMessage(content=state['question'])]
    }

def output_node(state: OverallState) -> dict:
    try:
        qa_chain = state['prompt'] | state['llm']
        result = qa_chain.invoke({'messages':state['messages']})

        return {
            **state,
            'messages':result
        }
    except Exception as e:
        print(f"❌ [OUTPUT NODE] Error: {e}")
        error_msg = AIMessage(content=f"Maaf, terjadi kesalahan: {str(e)}")

def summarizer_node(state: OverallState) -> dict:
    conversation = ''
    for msg in state['messages']:
        if isinstance(msg, HumanMessage):
            role = 'user'
        else:
            role = 'ai'
        conversation += f"{role}: {msg.content}\n\n"
    summarizer_prompt =f"""
        Buat ringkasan dari {conversation}, **maksimal 50 kata**. 
        Jelaskan inti permasalahan user, apa jawaban dari AI, dan apakah ada indikasi bahwa user butuh bantuan hukum lanjutan atau tidak to the point langsung ringkas tanpa header apapun.
        Jika user butuh bantuan dan user sudah konfirmasi, maka AI menjawab data dan permasalahan/kasus user akan di verifikasi oleh team dan akan dihubungi segera.
        """
    prompt = ChatPromptTemplate([
        ('system', summarizer_prompt)])
    qa_chain = prompt | state['llm']
    chat_summary = qa_chain.invoke({}).content
    return {
        **state,
        'summary': chat_summary
    }
    


builder = StateGraph(OverallState)
builder.add_node('retriever', retriever_node)
builder.add_node('merger', merger_node)
builder.add_node('output', output_node)    
builder.add_node('summarizer', summarizer_node)    
builder.add_edge(START, 'retriever')
builder.add_edge('retriever', 'merger')
builder.add_edge('merger', 'output')
builder.add_edge('output', 'summarizer')
builder.add_edge('summarizer', END)

checkpointer = InMemorySaver()
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}   
graph_memory = builder.compile(checkpointer = checkpointer)
graph = builder.compile()
# graph_memory = builder.compile()


In [282]:
# question = 'kalau gaji saya belum di bayarkan perusahaan, bisa saya kenakan pasal berapa aja, dan apa saja hak saya ?'
# question = 'saya bingung proses hukumnya gimana'
question = 'sudah saya hubungi via whatsapp, lalu apa ?'
result = graph_memory.stream({'question':question, 'llm':mistral_llm}, config, stream_mode = 'values')
# result = graph_memory.stream({'question':question, 'llm':mistral_llm})

In [283]:
for chunk in result:
    final_result = chunk

In [284]:
for i in final_result['messages']:
    i.pretty_print()

================================ Human Message =================================

kalau gaji saya belum di bayarkan perusahaan, bisa saya kenakan pasal berapa aja, dan apa saja hak saya ?
================================== Ai Message ==================================

Untuk gaji yang belum dibayarkan, Anda bisa mengajukan tuntutan berdasarkan Pasal 1320 KUHPerdata (KKP) tentang perjanjian kerja. Hak Anda meliputi:
1. Gaji yang belum dibayarkan.
2. Bunga gaji (jika ada ketidakpastian waktu pembayaran).
3. Ganti rugi jika ada kerugian tambahan.

Jika Anda bingung atau membutuhkan bantuan lebih lanjut, silakan hubungi kami melalui email [Watusi Legal Insight](contact@watusilegalinsight.com) atau WhatsApp 0812-3456-7890. Tim kami akan menghubungi Anda setelah verifikasi data dilakukan.
================================ Human Message =================================

saya bingung proses hukumnya gimana
================================== Ai Message ==================================

Untuk 

In [285]:
print(final_result['summary'])

User bingung tentang gaji yang belum dibayarkan, termasuk pasal hukum yang berlaku dan hak-haknya. AI menjelaskan pasal KUHPerdata yang relevan dan hak-hak user, serta langkah-langkah hukum yang bisa diambil. User kemudian bertanya tentang proses hukum, dan AI menjelaskan tahapan perundingan, pengadilan, dan bantuan hukum. User sudah menghubungi via WhatsApp, dan AI menunggu verifikasi data untuk memberikan bantuan lebih lanjut. User sudah konfirmasi, maka data dan permasalahan/kasus user akan di verifikasi oleh team dan akan dihubungi segera.


### Chatbot Function to Retrieve Question (chat bot with and without memory)

In [355]:
def chat_bot_with_memory(question:str, llm = None):
    if llm is None:
        llm = mistral_llm
        final_result = None

        for chunk in graph_memory.stream({'question': question, 'llm': llm}, config, stream_mode = 'values'):
            final_result = chunk
        for i in range(len(final_result['messages'])):
            final_result['messages'][i].pretty_print()
        return final_result, config['configurable']['thread_id']

def chat_bot(question: str, llm = None):
    if llm is None:
        llm = mistral_llm
    result = graph.invoke({'question':question, 'llm':llm})
    return result

In [290]:
question = 'bunyi pasal 1234'
result = chat_bot(question)
result

📋 [RETRIEVER NODE] Processing pasal: 1234


'Perikatan ditujukan untuk memberikan sesuatu, untuk berbuat sesuatu, atau untuk tidak berbuat sesuatu.'

In [291]:
question = 'Kantor saya tidak membayarkan upah selama 3 bulan terakhir, bisa dikenakan pasal berapa aja ?'
# question = 'Kalau 1602j bunyi pasalnya apa ?'
# question = 'siapa nama saya ?'
final_result, thread_id = chat_bot_with_memory(question)

🔍 [RETRIEVER NODE] No pasal found
================================ Human Message =================================

kalau gaji saya belum di bayarkan perusahaan, bisa saya kenakan pasal berapa aja, dan apa saja hak saya ?
================================== Ai Message ==================================

Untuk gaji yang belum dibayarkan, Anda bisa mengajukan tuntutan berdasarkan Pasal 1320 KUHPerdata (KKP) tentang perjanjian kerja. Hak Anda meliputi:
1. Gaji yang belum dibayarkan.
2. Bunga gaji (jika ada ketidakpastian waktu pembayaran).
3. Ganti rugi jika ada kerugian tambahan.

Jika Anda bingung atau membutuhkan bantuan lebih lanjut, silakan hubungi kami melalui email [Watusi Legal Insight](contact@watusilegalinsight.com) atau WhatsApp 0812-3456-7890. Tim kami akan menghubungi Anda setelah verifikasi data dilakukan.
================================ Human Message =================================

saya bingung proses hukumnya gimana
================================== Ai Message ========

## Rag Evaluation with RAGAS Framework

In [344]:
docs_inference = random.sample(docs, 5)

In [385]:
user_inputs = []
references = []

for doc in docs_inference:
    user_input = f"Bunyi pasal {doc.metadata.get('pasal')} apa dan terletak pada bab dan bagian berapa jika ada?"
    reference = doc.page_content
    
    user_inputs.append(user_input)
    references.append(reference)

responses = []
contexts = []

for user_input in user_inputs:
    result = chat_bot(user_input)

    responses.append(result['messages'][-1].content) 
    contexts.append([doc.page_content for doc in result['retrieved_docs']])

📋 [RETRIEVER NODE] Processing pasal: 1722
📋 [RETRIEVER NODE] Processing pasal: 1803
📋 [RETRIEVER NODE] Processing pasal: 1764
📋 [RETRIEVER NODE] Processing pasal: 1337
📋 [RETRIEVER NODE] Processing pasal: 1469


In [387]:
data = (
    {'user_input': user_inputs,
    'reference': references,
    'response': responses,
    'retrieved_contexts': contexts
    })

dataset = Dataset.from_dict(data)
dataset.save_to_disk('./outputs/generation/data_inference')

Saving the dataset (1/1 shards): 100%|██████████| 5/5 [00:00<00:00, 685.90 examples/s]


In [400]:
evaluator_llm = LangchainLLMWrapper(mistral_llm)
result = evaluate(dataset=dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)

Evaluating: 100%|██████████| 15/15 [00:14<00:00,  1.05it/s]


In [401]:
df_result = result.to_pandas()
df_result.to_csv('outputs\generation\evaluation\mistral_score.csv', index = False)

In [402]:
print(result)
df_result

{'context_recall': 1.0000, 'faithfulness': 1.0000, 'factual_correctness(mode=f1)': 0.9340}


,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,Bunyi pasal 1722 apa dan terletak pada bab dan...,[Bab: BAB XI - PENITIPAN BARANG\nBagian: BAGIA...,"Bunyi Pasal 1722: ""Jika pemberi titipan bergan...",Bab: BAB XI - PENITIPAN BARANG\nBagian: BAGIAN...,1.0,1.0,1.00
1,Bunyi pasal 1803 apa dan terletak pada bab dan...,[Bab: BAB XVI - PEMBERIAN KUASA\nBagian: BAGIA...,Bunyi Pasal 1803:\nPenerima kuasa bertanggung ...,Bab: BAB XVI - PEMBERIAN KUASA\nBagian: BAGIAN...,1.0,1.0,1.00
2,Bunyi pasal 1764 apa dan terletak pada bab dan...,[Bab: BAB XIII - PINJAM PAKAI HABIS\nBagian: B...,"Bunyi Pasal 1764: ""Jika ia tidak mungkin memen...",Bab: BAB XIII - PINJAM PAKAI HABIS\nBagian: BA...,1.0,1.0,1.00
3,Bunyi pasal 1337 apa dan terletak pada bab dan...,[Bab: BAB II - PERIKATAN YANG LAHIR DARI KONTR...,"Bunyi Pasal 1337: ""suatu sebab adalah terlaran...",Bab: BAB II - PERIKATAN YANG LAHIR DARI KONTRA...,1.0,1.0,0.67
4,Bunyi pasal 1469 apa dan terletak pada bab dan...,[Bab: BAB V - JUAL BELI\nBagian: BAGIAN 1 - Ke...,"Bunyi Pasal 1469: ""Atas ancaman yang sama, par...",Bab: BAB V - JUAL BELI\nBagian: BAGIAN 1 - Ket...,1.0,1.0,1.00
